This is a custom written code to compute the class activation maps (CAM), gradient weighted class activation maps (grad-CAM), average CAM and average grad-CAM for the class of our interest. The code also prints the pixel values of the heat maps computed for every image and the average value of the heat maps for the given class. It also prints the results of classification. The model architecture used in this study has a global average pooling layer(GAP) immediately following the deepest convolutional layer. The GAP layer is followed by the final dense layer with two neurons for the two categories. This code needs to be modified to suit your model requirements. 

In [ ]:
#load libraries
from __future__ import print_function
from keras.models import model_from_json
from keras.models import load_model
from keras import backend as K
from keras.preprocessing.image import img_to_array
from scipy import ndimage
from skimage.measure import regionprops
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import cv2
import os
import glob
%matplotlib inline

In [ ]:
#get layer information from the model
def get_output_layer(model, layer_name):
    try:
        layer_dict = dict([(layer.name, layer) for layer in model.layers]) 
        layer = layer_dict[layer_name]
        return layer
    except Exception as e:
        raise Exception('Error from get_output_layer(): ' + str(e))

In [ ]:
#generate class activation maps (CAM and grad-CAM) for discriminative localization

def Generate_CAM (thisModel, thisImg_path):
    try:
        
        # preprocess input image      
        original_img = cv2.imread(thisImg_path)
        resized_original_image = cv2.resize(original_img, (512, 512)) #size of input
        input_image = img_to_array(resized_original_image) 
        width, height, _ = input_image.shape
        input_image = np.array(input_image, dtype="float") /255.0        
        input_image = input_image.reshape((1,) + input_image.shape)
        class_weights = thisModel.layers[-1].get_weights()[0] 
        get_output = K.function([thisModel.layers[0].input], 
                                [thisModel.layers[-3].output, thisModel.layers[-1].output]) 
        [conv_outputs, predictions] = get_output([input_image])
        conv_outputs = conv_outputs[0, :, :, :]     
        final_output = predictions   
        
        #Create the class activation map (CAM).
        cam = np.zeros(dtype = np.float32, shape = conv_outputs.shape[0:2])     
        grad_cam = np.zeros(dtype = np.float32, shape = conv_outputs.shape[0:2])
        
        for i, w in enumerate(class_weights[:, 1]): 
            cam += w * conv_outputs[:, :, i]            

        grad_cam = cam              

        cam /= np.max(cam)                                      
        resized_cam = cv2.resize(cam, (height, width)) # upscale to input image size

        # Create heatmap based on CAM
        cam_heatmap = cv2.applyColorMap(np.uint8(255*resized_cam), cv2.COLORMAP_JET)
        cam_heatmap[np.where(resized_cam < 0.2)] = 0  
        cam_img = np.float32(cam_heatmap) + np.float32(resized_original_image)  
        cam_img = 255 * cam_img / np.max(cam_img) 

        #Create the gradient-weighted class activation map (grad-CAM). 
        grad_cam = np.maximum(grad_cam, 0.)  # applying ReLU 
        grad_cam /= np.max(grad_cam)                    
        resized_grad_cam = cv2.resize(grad_cam, (height, width))   # upscaling to input image size

        # Create heatmap based on grad-CAM
        grad_heatmap = cv2.applyColorMap(np.uint8(255*resized_grad_cam), cv2.COLORMAP_JET)
        grad_img = np.float32(grad_heatmap) + np.float32(resized_original_image)  
        grad_img = 255 * grad_img / np.max(grad_img)

        return[final_output, cam, cam_img, grad_cam, grad_img, plt, resized_cam, resized_grad_cam]
    except Exception as e:
        raise Exception('Error from Generate_CAM(): ' + str(e))

In [ ]:
def Load_Pretrained_Model(thisModel_path, thisModel_name):
    try:
        loaded_model = load_model(os.path.join(thisModel_path, thisModel_name + '.hdf5')) 
        return(loaded_model)
    except Exception as e:
        raise Exception('Error from Load_Pretrained_Model(): ' + str(e))
try:
    # Load Keras model
    print("Loading Keras model...")
    try:
        thisLoaded_Model = Load_Pretrained_Model('/home/rajaramans2/codes/app_sciences/',
                                                 'weights-improvement-39-0.9615') #path to your model
        
    except Exception as e:
        raise Exception('Error in loading CNN models: ' + str(e))
    
    try:
        CAM_output_path      = 'CAM' 
        gradCAM_output_path  = 'gradCAM'
        print("Set up output folder: " + CAM_output_path)
        f1 = open(os.path.join(CAM_output_path, 'ClassificationResult.txt'), 'w')
    except Exception as e:
        raise Exception('Error in saving classification result: ' + str(e))
        
    try:
        TestFolder_path = '/home/rajaramans2/codes/app_sciences/cxr_pneumonia_512/test/pneumonia/' #modify the classes to visualize
        print ("Input image folder: " + TestFolder_path)
        Test_image_path = TestFolder_path + '*.jpg'
        Test_image_Set = glob.glob(Test_image_path)
    except Exception as e:
        raise Exception('Error in retrieving images from dataset folder: ' + str(e))   
 
    resized_avg_CAM     = np.zeros(dtype = np.float32, shape =thisLoaded_Model.input.shape[1:3]) 
    resized_avg_gradCAM = np.zeros(dtype = np.float32, shape =thisLoaded_Model.input.shape[1:3])
    avg_CAM             = np.zeros(dtype = np.float32, shape =thisLoaded_Model.layers[-3].output.shape[1:3])
    avg_gradCAM         = np.zeros(dtype = np.float32, shape =thisLoaded_Model.layers[-3].output.shape[1:3])

    i = 0
    err_cnt = 0
    for Img_filename in Test_image_Set:
        OutScores, aCAM, aCAM_Img, aGradCAM, aGradCAM_Img, thisPlt, r_CAM, r_gradCAM = Generate_CAM(thisLoaded_Model, Img_filename)
                  
        fname = os.path.basename(Img_filename)
        name_only, ext_only = os.path.splitext(fname)

        aCAM_file       = name_only         
        aGradCAM_File   = name_only        

        aCAM_ImgFile    = name_only         
        aGradCAM_ImgFile = name_only

        bbox_file       = name_only         
     
        f1.write("{}:    [{:.3f},  {:.3f}]".format(fname, OutScores[0][0], OutScores[0][1])) 
        target = OutScores[0][1] 
        TP = True                                                                                                                            
        for other_output in OutScores[0]:
            if target < other_output:       
                err_cnt += 1

                aCAM_file       += "_CAM_err.txt"
                aGradCAM_File   += "_gradCAM_err.txt"

                aCAM_ImgFile    += "_CAM_err.png"
                aGradCAM_ImgFile += "_gradCAM_err.png"
                bbox_file       += "_bbox_err.png"

                f1.write("  ERR\n")
                TP = False
                break
            
        if TP:
            resized_avg_CAM += r_CAM         # create average CAM using CAM for correctly classified input images
            avg_CAM += aCAM                  # average CAM for original CAM data 
            resized_avg_gradCAM += r_gradCAM         
            avg_gradCAM += aGradCAM                  

            aCAM_file       += "_CAM.txt"
            aGradCAM_File   += "_gradCAM.txt"

            aCAM_ImgFile    += "_CAM.png"
            aGradCAM_ImgFile += "_gradCAM.png"
            bbox_file       += "_bbox.png"

            f1.write("\n")

        f2 = open(os.path.join(CAM_output_path, aCAM_file), 'w')
        row, col = aCAM.shape
        for n in range(row):
            for m in range(col):
                f2.write("{:2.4f}  ".format(aCAM[n][m])) 
            f2.write("\n")
        row, col = r_CAM.shape
        for n in range(row):
            for m in range(col):
                f2.write("{:2.4f}  ".format(r_CAM[n][m]))
            f2.write("\n")
        f2.close()
        
        f3 = open(os.path.join(gradCAM_output_path, aGradCAM_File), 'w')
        row, col = aGradCAM.shape
        for n in range(row):
            for m in range(col):
                f3.write("{:2.4f}  ".format(aGradCAM[n][m]))
            f3.write("\n")
        row, col = r_gradCAM.shape
        for n in range(row):
            for m in range(col):
                f3.write("{:2.4f}  ".format(r_gradCAM[n][m]))
            f3.write("\n")
        f3.close()
       
        
        cv2.imwrite(os.path.join(CAM_output_path, aCAM_ImgFile), aCAM_Img)
        cv2.imwrite(os.path.join(gradCAM_output_path, aGradCAM_ImgFile), aGradCAM_Img)

        thisPlt.close()

        i += 1
                                                                                              
    f1.write("\n Total Errors: {:d}\n".format(err_cnt))
    f1.write("No. of testing images: {:d}\n".format(i))
    f1.write("Accuracy: {:.5f}\n".format((float(i-err_cnt)/float(i))*100.))
    f1.close()
  
    avg_CAM /= np.max(avg_CAM)
    resized_avg_CAM /= np.max(resized_avg_CAM)

    f4 = open(os.path.join(CAM_output_path, 'avg_CAM.txt'), 'w')
    row, col = avg_CAM.shape
    for n in range(row):
        for m in range(col):
            f4.write("{:2.4f}  ".format(avg_CAM[n][m])) 
        f4.write("\n")
    row, col = resized_avg_CAM.shape
    for n in range(row):
        for m in range(col):
            f4.write("{:2.4f}  ".format(resized_avg_CAM[n][m]))
        f4.write("\n")
    f4.close()
    
    resized_avg_heatmap = cv2.applyColorMap(np.uint8(255*resized_avg_CAM), cv2.COLORMAP_JET)
    resized_avg_heatmap[np.where(resized_avg_CAM < 0.2)] = 0
    resized_avg_img = np.float32(resized_avg_heatmap)       
    resized_avg_img = 255 * resized_avg_img / np.max(resized_avg_img)
    cv2.imwrite(os.path.join(CAM_output_path, 'avg_CAM.png'), resized_avg_img)

    avg_gradCAM /= np.max(avg_gradCAM)
    resized_avg_gradCAM /= np.max(resized_avg_gradCAM)

    f5 = open(os.path.join(gradCAM_output_path, 'avg_gradCAM.txt'), 'w')
    row, col = avg_gradCAM.shape
    for n in range(row):
        for m in range(col):
            f5.write("{:2.4f}  ".format(avg_gradCAM[n][m])) 
        f5.write("\n")
    row, col = resized_avg_gradCAM.shape
    for n in range(row):
        for m in range(col):
            f5.write("{:2.4f}  ".format(resized_avg_gradCAM[n][m]))
        f5.write("\n")
    f5.close()
  
    resized_avg_gradHeatmap = cv2.applyColorMap(np.uint8(255*resized_avg_gradCAM), cv2.COLORMAP_JET)
    resized_avg_gradImg = np.float32(resized_avg_gradHeatmap)       
    resized_avg_gradImg = 255 * resized_avg_gradImg / np.max(resized_avg_gradImg)
    cv2.imwrite(os.path.join(gradCAM_output_path, 'avg_gradCAM.png'), resized_avg_gradImg)
    
    print (i)
    print('done')
    
except Exception as e:
    print (str(e))